# Microcosm analysis 16s updated

In [ ]:
!qiime --version

# 2019 7

In [ ]:
from metaBarTools import metaBar_PreX
import os

In [ ]:
metaBar = metaBar_PreX()

In [ ]:
platesetup = os.path.abspath("./metadata/platesetup_all.xlsx")
print(platesetup)

In [ ]:
m_presoil = os.path.abspath("./rename_reads/modified_names/reads_presoil/")
m_post2 = os.path.abspath("./rename_reads/modified_names/reads_post2/")
m_post4 = os.path.abspath("./rename_reads/modified_names/reads_post4/")
m_post5 = os.path.abspath("./rename_reads/modified_names/reads_post5/")

In [ ]:
print(m_presoil, m_post2, m_post4, m_post5, sep="\n")

In [ ]:
manifest = metaBar.metaBar_Qiime2_Manifest(m_presoil, m_post2, m_post4, m_post5, platesetup, colnames = ['Plate', 'Sample_ID', 'Sample_label', 'PCR_Conc', 'nmol_per_sample', 'Amount_of_Sample', 'Amount_of_Water', 'Well_No', 'Primer_set'], paired=True, matchby="sample")

In [ ]:
metadatapath = os.path.abspath("./metadata/metadata_mcm_16s_official.v2.tsv")
print(metadatapath)

In [ ]:
# path_16s, path_ITS = metaBar.metaBar_makeSubDir("Analysis/updated_mcm", ["16S_result", "ITS2_result"])
path_16s = "./running_project/2019/soil_project/micocosm/Microcosm/Analysis/updated_mcm/16S_result"

path_ITS = "./running_project/2019/soil_project/micocosm/Microcosm/Analysis/updated_mcm/ITS2_result"

In [ ]:
print(path_16s, path_ITS, sep="\n")

In [ ]:
man_16s = os.path.abspath("./manifest_mcm/16SF@16SR_manifest.csv")

In [ ]:
os.chdir(path_16s)
os.getcwd()

### import

In [ ]:
!qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path $man_16s \
--output-path mcm_16s_raw_seq.qza \
--input-format PairedEndFastqManifestPhred33

In [ ]:
!qiime demux summarize \
--i-data mcm_16s_raw_seq.qza \
--o-visualization mcm_16s_raw_seq.qzv

In [ ]:
mcm_16s_F = len("CCTACGGGNGGCWGCAG")
mcm_16S_R = len("GGACTACHVGGGTATCTAATCC")

In [ ]:
if not os.path.exists("feature-tables"):
    os.makedirs("feature-tables")
    
if not os.path.exists("dada2_stats"):
    os.makedirs("dada2_stats")

In [ ]:
#298
#258
#------
#289
#257
#------
#289
#247
#-----
#289
#220
!qiime dada2 denoise-paired \
--i-demultiplexed-seqs mcm_16s_raw_seq.qza \
--o-table feature-tables/table_mcm_16s \
--o-representative-sequences rep_mcm_16s \
--p-trim-left-f $mcm_16s_F \
--p-trim-left-r $mcm_16S_R \
--p-trunc-len-f 289 \
--p-trunc-len-r 220 \
--o-denoising-stats dada2_stats/dada2_stats_mcm_16s.qza \
--p-n-threads 12

In [ ]:
!qiime metadata tabulate \
--m-input-file dada2_stats/dada2_stats_mcm_16s.qza \
--o-visualization dada2_stats/stats-dada2.qzv

## phylogenics (make tree)

In [ ]:
!qiime tools export \
--input-path rep_mcm_16s.qza \
--output-path rep_mcm_16s_seq

**alignment mafft ecounter memory error** 
> use --p-parttree flag

In [ ]:
!qiime alignment mafft \
--i-sequences rep_mcm_16s.qza \
--o-alignment aligned_rep_seqs.qza \
--p-parttree \
--p-n-threads 16

In [ ]:
!qiime alignment mask \
--i-alignment aligned_rep_seqs.qza \
--o-masked-alignment masked_aligned_rep_seqs.qza

In [ ]:
!qiime phylogeny fasttree \
--i-alignment masked_aligned_rep_seqs.qza \
--o-tree unrooted_tree.qza

In [ ]:
# root the tree
!qiime phylogeny midpoint-root \
--i-tree unrooted_tree.qza \
--o-rooted-tree rooted_tree.qza

---

In [ ]:
# use sklearn NB classifier
if not os.path.exists("classifier"):
    os.makedirs("classifier")

In [ ]:
db_seq_path = "/Database/silva_132_release_08102019/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna"
tax_path = "/Database/silva_132_release_08102019/SILVA_132_QIIME_release/taxonomy/16S_only/99/trimmed_taxonomy_7_levels.txt"

In [ ]:
!qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path $db_seq_path \
--output-path ./classifier/silva_132_99.qza

!qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-format HeaderlessTSVTaxonomyFormat \
--input-path $tax_path \
--output-path ./classifier/taxonomy_silva_132_99.qza

In [ ]:
!qiime feature-classifier extract-reads \
--i-sequences ./classifier/silva_132_99.qza \
--p-f-primer CCTACGGGNGGCWGCAG \
--p-r-primer GACTACHVGGGTATCTAATCC \
--p-min-length 100 \
--p-max-length 460 \
--o-reads ./classifier/ref_silva_132_99.qza

In [ ]:
!qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ./classifier/ref_silva_132_99.qza \
--i-reference-taxonomy ./classifier/taxonomy_silva_132_99.qza \
--o-classifier ./classifier/classifier_silva_132.qza

In [ ]:
if not os.path.exists("taxonomy"):
    os.makedirs("taxonomy")

In [ ]:
!qiime feature-classifier classify-sklearn \
--i-classifier ./classifier/classifier_silva_132.qza \
--i-reads rep_mcm_16s.qza \
--o-classification ./taxonomy/taxonomy_mcm_16s.qza

In [ ]:
!qiime metadata tabulate \
--m-input-file ./taxonomy/taxonomy_mcm_16s.qza \
--o-visualization ./taxonomy/taxonomy_mcm_16s.qzv

In [ ]:
!qiime taxa barplot \
--i-table feature-tables/table_mcm_16s.qza \
--i-taxonomy taxonomy/taxonomy_mcm_16s.qza \
--m-metadata-file $metadatapath \
--o-visualization taxonomy/barplot_mcm_16s.qzv

### export for decontam

In [ ]:
# !qiime tools export \
# --input-path feature-tables/table_mcm_16s.qza \
# --output-path exported

In [ ]:
# !qiime tools export \
# --input-path taxonomy/taxonomy_mcm_16s.qza \
# --output-path exported/

In [ ]:
# !cp exported/taxonomy.tsv exported/biom-taxonomy.tsv

In [ ]:
# # change header
# !sed -i '1 s/Feature ID/#OTUID/g; s/Taxon/taxonomy/g; s/Confidence/confidence/g' exported/biom-taxonomy.tsv

In [ ]:
# !biom add-metadata \
# -i exported/feature-table.biom \
# -o exported/feature-table-tax.biom \
# --observation-metadata-fp exported/biom-taxonomy.tsv \
# --sample-metadata-fp $metadatapath \
# --sc-separated taxonomy

In [ ]:
# !biom convert \
# -i exported/feature-table-tax.biom \
# -o exported/feature-table.tsv \
# --to-tsv

> process with decontam_mcm_16s.R, we use frequency

In [ ]:
# !qiime feature-table filter-features \
# --i-table feature-tables/table_mcm_16s.qza \
# --m-metadata-file exported/contam_by_freq.txt \
# --p-exclude-ids True \
# --o-filtered-table exported/decontam_filtered_table.qza

In [ ]:
# !qiime taxa filter-table \
# --i-table exported/decontam_filtered_table.qza \
# --i-taxonomy taxonomy/taxonomy_mcm_16s.qza \
# --p-exclude mitochondria,chloroplast \
# --o-filtered-table feature-tables/nochloroplast_table.qza

In [ ]:
!qiime taxa filter-table \
--i-table feature-tables/table_mcm_16s.qza \
--i-taxonomy taxonomy/taxonomy_mcm_16s.qza \
--p-exclude mitochondria,chloroplast \
--o-filtered-table feature-tables/nochloroplast_table.qza

### separate control groups

In [ ]:
# remove controls
!qiime feature-table filter-samples \
--i-table feature-tables/nochloroplast_table.qza \
--m-metadata-file $metadatapath \
--p-where "cul_type='Control'" \
--p-exclude-ids \
--o-filtered-table feature-tables/filtered_table_nocontrols.qza

In [ ]:
# get controls
!qiime feature-table filter-samples \
--i-table feature-tables/table_mcm_16s.qza \
--m-metadata-file $metadatapath \
--p-where "cul_type='Control'" \
--o-filtered-table feature-tables/feature_table_controls.qza

### export for R

In [ ]:
!qiime tools export \
--input-path feature-tables/filtered_table_nocontrols.qza \
--output-path R_process/

In [ ]:
!qiime tools export \
--input-path taxonomy/taxonomy_mcm_16s.qza \
--output-path R_process/

In [ ]:
!cp R_process/taxonomy.tsv R_process/biom-taxonomy.tsv

In [ ]:
# change header
!sed -i '1 s/Feature ID/#OTUID/g; s/Taxon/taxonomy/g; s/Confidence/confidence/g' R_process/biom-taxonomy.tsv

In [ ]:
metadatapath

In [ ]:
# updated rotation infor for WA based on WSU research
!biom add-metadata \
-i R_process/feature-table.biom \
-o R_process/feature-table-tax.biom \
--observation-metadata-fp R_process/biom-taxonomy.tsv \
--sample-metadata-fp $metadatapath \
--sc-separated taxonomy

In [ ]:
!biom convert \
-i R_process/feature-table-tax.biom \
-o R_process/feature-table.tsv \
--to-tsv

In [ ]:
!qiime tools export \
--input-path rooted_tree.qza \
--output-path R_process/

### alpha rarefaction curves

In [ ]:
!qiime diversity alpha-rarefaction \
--i-table feature-tables/table_mcm_16s.qza \
--i-phylogeny rooted_tree.qza \
--m-metadata-file $metadatapath \
--p-steps 100 \
--p-max-depth 15000 \
--o-visualization alpha-rarecurve

*End*: move to R for analysis